### Imports

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import urllib.parse

### HTTP Requests

In [50]:
#store website link as a variable
url = 'https://www.yelp.com/search?find_desc=restaurants&find_loc=Seattle%2C%20WA&start=0'

#get requests
response = requests.get(url)

#check status code
print(response.status_code)

#convert to soup object
soup = bs(response.content, 'html.parser')
#print(soup.prettify())

200


### Results

In [340]:
results = soup.find_all('div', {'class':'container__09f24__sxa9-'})
print(len(results))

10


### Target necessary data 

In [ ]:
# name
# stars
# stars_count
# price
# area
# restaurant type
# services

Name

In [205]:
# Get name
results[0].find('a', {'class': 'css-166la90'}).get('name')

'The Pink Door'

Star

In [206]:
results[0].find('div', {'class':'i-stars__09f24___sZu0'}).get('aria-label')

'4.5 star rating'

Star count

In [207]:
results[0].find('span', {'class':'reviewCount__09f24__3GsGY'}).get_text()
#result[0].find('span', {'class':'css-e81eai'})

'5469'

Price

In [208]:
results[0].find('span', {'class':'priceRange__09f24__2GspP'}).get_text()

'$$'

Area

In [209]:
results[0].find(lambda tag: tag.name == 'span' and tag.get('class') == ['css-e81eai']).get_text()

'Downtown'

Restaurant type

In [210]:
results[0].find_all('p', {'class': 'text__09f24__2NHRu'})

[<p class="text__09f24__2NHRu css-1hx6l2b">Italian</p>,
 <p class="text__09f24__2NHRu css-1hx6l2b">Wine Bars</p>,
 <p class="text__09f24__2NHRu css-1hx6l2b">Seafood</p>]

In [211]:
restaurant_type=[]

# Loop through results
for rtype in results:
    rlist = rtype.find_all('p', {'class': 'text__09f24__2NHRu'})
    data = [rtype.get_text() for rtype in rlist]
    restaurant_type.append(data)

In [212]:
restaurant_type

[['Italian', 'Wine Bars', 'Seafood'],
 ['Tapas/Small Plates', 'Cocktail Bars', 'Breakfast & Brunch'],
 ['Breakfast & Brunch', 'Coffee & Tea', 'Southern'],
 ['Southern', 'Cocktail Bars'],
 ['American (New)', 'Cocktail Bars'],
 ['Seafood', 'Tapas/Small Plates'],
 ['American (New)', 'Seafood', 'Cocktail Bars'],
 ['American (New)', 'Breakfast & Brunch', 'Desserts'],
 ['Seafood'],
 ['Tapas/Small Plates', 'Cocktail Bars', 'American (Traditional)']]

Services

In [213]:
results[0].find_all('span', {'class': 'raw__09f24__3Azrj'})

[<span class="raw__09f24__3Azrj">Proof of vaccination required</span>,
 <span class="raw__09f24__3Azrj">Delivery</span>,
 <span class="raw__09f24__3Azrj">Takeout</span>]

In [301]:
restaurant_services=[]

# Loop through results
for rservice in results:
    rlist = rservice.find_all('span', {'class': 'raw__09f24__3Azrj'})
    data = [r.get_text() for r in rlist]
    restaurant_services.append(data)

In [302]:
restaurant_services

[['Delivery', 'Takeout'],
 ['Outdoor dining', 'Delivery', 'Takeout'],
 ['Delivery'],
 ['Outdoor dining', 'Delivery', 'Takeout'],
 ['Outdoor dining', 'Delivery', 'Takeout'],
 ['Outdoor dining', 'Delivery', 'Takeout'],
 ['Proof of vaccination required', 'Delivery', 'Takeout'],
 ['Outdoor dining', 'Delivery', 'Takeout'],
 ['Live wait time: 0 mins', 'Outdoor dining', 'Delivery', 'Takeout'],
 ['Outdoor dining', 'Takeout']]

Append all Results in Lists with List Comprehension

In [238]:
# name
# stars
# stars_count
# price
# area
name = [result.find('a', {'class': 'css-166la90'}).get('name') for result in results]
stars = [result.find('div', {'class':'i-stars__09f24___sZu0'}).get('aria-label') for result in results]
stars_count = [result.find('span', {'class':'reviewCount__09f24__3GsGY'}).get_text() for result in results]
price = [result.find('span', {'class':'priceRange__09f24__2GspP'}).get_text() for result in results]
area = [result.find(lambda tag: tag.name == 'span' and tag.get('class') == ['css-e81eai']).get_text() for result in results]

### Create Pandas Data Frame

In [239]:
yelp_first_page = pd.DataFrame({'Name': name,
                           'Star': stars,
                           'Stars_count': stars_count,
                           'Price': price,
                           'Area': area,
                           'Types': restaurant_type,
                           'Services': restaurant_services})

In [240]:
yelp_first_page

,Name,Star,Stars_count,Price,Area,Types,Services
0,Pomodoro,4.5 star rating,494,$$,Eastlake,"[Italian, Spanish, Tapas/Small Plates]","[Proof of vaccination required, Delivery, Take..."
1,Fogón Cocina Mexicana,4.5 star rating,1601,$$,Capitol Hill,[Mexican],"[Live wait time: 0 mins, Outdoor dining, Deliv..."
2,LOCAL Public Eatery,4 star rating,414,$$,South Lake Union,"[Bars, American (New)]","[Outdoor dining, Delivery, Takeout]"
3,G.H. Pasta Co.,4.5 star rating,241,$$,Denny Triangle,"[Italian, Pasta Shops]","[Delivery, Takeout]"
4,Duke’s Seafood,4 star rating,1001,$$,South Lake Union,"[Seafood, Bars]","[Delivery, Takeout]"
5,Bar Harbor,4 star rating,237,$$,South Lake Union,"[Seafood, Cocktail Bars]","[Outdoor dining, Delivery, Takeout]"
6,Meet Korean BBQ,4.5 star rating,149,$$$,Capitol Hill,"[Barbeque, Korean]","[Outdoor dining, Delivery, Takeout]"
7,Reckless Noodle House,4.5 star rating,263,$$,Atlantic,"[Bars, Noodles, Asian Fusion]","[Outdoor dining, Delivery]"
8,Von’s 1000 Spirits,4.5 star rating,1943,$$,Downtown,"[American (New), Burgers, Pubs]","[Outdoor dining, Delivery, Takeout]"
9,Zig Zag Cafe,4.5 star rating,1037,$$,Downtown,"[American (New), Cocktail Bars, Mediterranean]",[Proof of vaccination required]


### Multiple Pages

In [507]:
# create empty dataframe
Restaurants = pd.DataFrame(columns=['Name', 'Star', 'Stars_count', 'Price', 'Area', 'Category', 'Services'])

# scrape 3 pages

# sequence of numbers (1 up 3, 4 is not included)
for i in range(0,25):
    # website request
    website = requests.get('https://www.yelp.com/search?find_desc=restaurants&find_loc=Federal%20Way%2C%20WA&start=' + str(i) + '0')
    
    # create soup object
    soup = bs(website.content, 'html.parser')

    # result items
    results = soup.find_all('div', {'class':'container__09f24__sxa9-'})
    
    category=[]
    services=[]
    # Lists
    for result in results:
        try:
            name = result.find('a', {'class': 'css-166la90'}).get('name')
        except:
            name = 'n/a'
   
        try:
            stars = result.find('div', {'class':'i-stars__09f24___sZu0'}).get('aria-label').replace(' star rating', '')
        except:
            stars = 'n/a'
            
        try:
            stars_count = result.find('span', {'class':'reviewCount__09f24__3GsGY'}).get_text() 
        except:
            stars_count = 'n/a'
            
        try:
            price = result.find('span', {'class':'priceRange__09f24__2GspP'}).get_text()
        except:
            price = 'n/a'

        try:
            area = result.find(lambda tag: tag.name == 'span' and tag.get('class') == ['css-e81eai']).get_text()
        except Exception as e:
            area = 'n/a'
    
    # Loop through category
        try:
            rlist = result.find_all('p', {'class': 'text__09f24__2NHRu'})
            category = [result.get_text() for result in rlist]
        except:
            category = 'n/a'
    
    # Loop through services
        try:
            rlist = result.find_all('span', {'class': 'raw__09f24__3Azrj'})
            services = [result.get_text() for result in rlist]
        except:
            services = 'n/a'
    
        Restaurants = Restaurants.append({'Name': name,
                                          'Star': stars,
                                          'Stars_count': stars_count,
                                          'Price': price,
                                          'Area': area,
                                          'Category': category,
                                          'Services': services
                                          },
                                           ignore_index=True)

In [501]:
Restaurants

,Name,Star,Stars_count,Price,Area,Category,Services
0,Blackbird,4.5,20,n/a,,"[American (New), Diners, Breakfast & Brunch]",[Outdoor dining]
1,Drumlin,5,7,n/a,,"[Bars, Performing Arts, American (Traditional)]","[Outdoor dining, Takeout]"
2,The Market,4.5,398,$$,,[Seafood],"[Outdoor dining, Delivery, Takeout]"
3,Royal Dumplings House,4.5,22,n/a,,[Chinese],"[Outdoor dining, Delivery, Takeout]"
4,Fire & the Feast,4,60,n/a,,[Italian],"[Outdoor dining, Delivery, Takeout]"
...,...,...,...,...,...,...,...
235,Flying Squirrel Pizza,4,173,$$,Mapleleaf,[Pizza],[Takeout]
236,Baja Tacos & Beer Mexican Restaurant,4,161,$$,Crown Hill,[Mexican],"[Delivery, Takeout]"
237,Santa Fe Mexican Grill & Cantina,3.5,65,$$,,"[Mexican, Beer Bar, Cocktail Bars]","[Outdoor dining, Delivery, Takeout]"
238,Burgermaster,4,255,$,Licton Springs,[Burgers],"[Delivery, Takeout]"


In [313]:
#Restaurants_Seattle.to_csv('Restaurants_Seattle.csv', index=False, encoding='utf-8-sig', na_rep='Unknown')

In [493]:
#Restaurants.to_csv('Restaurants.csv', index=False, encoding='utf-8-sig', na_rep='Unknown')

In [508]:
with open('Restaurants.csv', 'a') as f:
    Restaurants.to_csv(f, header=False, index=False, encoding='utf-8-sig', na_rep='Unknown')

In [407]:
Restaurants_Seattle.duplicated('Name')

0      False
1      False
2      False
3      False
4      False
       ...  
235    False
236     True
237    False
238    False
239    False
Length: 240, dtype: bool

In [327]:
Restaurants_Seattle['Name'].value_counts()

Tacos Chukis                     3
Taylor Shellfish Oyster Bar      3
Finch & Pine                     2
Kona Kitchen - Seattle           2
Habitant                         2
                                ..
Portage Bay Cafe - Roosevelt     1
Charlotte Restaurant & Lounge    1
Kokkaku                          1
Star Fusion & Bar                1
Rondo                            1
Name: Name, Length: 208, dtype: int64

### Work Hours

Get final URL by combining two parts

URL Part 1

In [411]:
# we combine url part no.1 with url part no.2 in order to get the final URL
url_part_1 = 'https://www.yelp.com/'

Create List for URL Part 2

In [413]:
url_part_2 = []

# Loop through results
for item in results:
    # Loop through links
    for link in item.find_all('a', {'class': 'css-166la90'}):
        url_part_2.append(link.get('href'))

In [414]:
len(url_part_2)

10

Join URL 1 and URL 2

In [415]:
url_joined = []

for link_2 in url_part_2:
    url_joined.append(urllib.parse.urljoin(url_part_1, link_2))

In [416]:
url_joined

['https://www.yelp.com/biz/the-pink-door-seattle-4?osq=restaurants',
 'https://www.yelp.com/biz/nue-seattle?osq=restaurants',
 'https://www.yelp.com/biz/biscuit-bitch-seattle?osq=restaurants',
 'https://www.yelp.com/biz/2120-seattle?osq=restaurants',
 'https://www.yelp.com/biz/witness-seattle?osq=restaurants',
 'https://www.yelp.com/biz/the-walrus-and-the-carpenter-seattle?osq=restaurants',
 'https://www.yelp.com/biz/six-seven-restaurant-seattle-3?osq=restaurants',
 'https://www.yelp.com/biz/list-seattle?osq=restaurants',
 'https://www.yelp.com/biz/tilikum-place-cafe-seattle-3?osq=restaurants',
 'https://www.yelp.com/biz/elliotts-oyster-house-seattle-2?osq=restaurants']

#### Get Hours Data from the First Link

In [417]:
# Store the first link as a variable
first_link = url_joined[0]

In [418]:
response_1 = requests.get(first_link)
soup_1 = bs(response_1.content, 'html.parser')

In [ ]:
# Name
# Address 
# Work Hours
# Reviews

Name

In [419]:
soup_1.find('h1', {'class': 'css-11q1g5y'}).get_text()

'The Pink Door'

Address

In [420]:
soup_1.find('address').get_text(", ")

'1919 Post Alley, Seattle, WA 98101'

Schedule

In [421]:
WeekDay = soup_1.find('tbody').find_all('th')
Week = [week.string for week in WeekDay]
Week

['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [511]:
Hours = soup_1.find('tbody').find_all('tr')
Hours = Hours[1::2]

In [513]:
hour = []
for tr in Hours:
    td = tr.find_all('td')
    data = [str(tr.get_text(", ")).strip('') for tr in td]
    hour.append(data)

In [514]:
hour

[['Closed', ''],
 ['Closed', ''],
 ['11:30 AM - 3:30 PM, 5:00 PM - 10:00 PM', ''],
 ['11:30 AM - 3:30 PM, 5:00 PM - 10:00 PM', ''],
 ['11:30 AM - 3:30 PM, 5:00 PM - 10:00 PM', 'Open now'],
 ['11:30 AM - 3:30 PM, 5:00 PM - 10:00 PM', ''],
 ['11:30 AM - 3:30 PM, 5:00 PM - 10:00 PM', '']]

In [457]:
#hours
try:
    hours = soup_1.find('tbody').find_all('tr')[1::2]
    hour = []
    for tr in hours:
        td = tr.find_all('td')
        data = [str(tr.get_text(", ")).strip('') for tr in td]
        hour.append(data)
except:
    hour[0] = 'n/a',
    hour[1] = 'n/a',
    hour[2] = 'n/a',
    hour[3] = 'n/a',
    hour[4] = 'n/a',
    hour[5] = 'n/a',
    hour[6] = 'n/a'

In [458]:
hour[6]

['11:30 AM - 3:30 PM, 5:00 PM - 10:00 PM', '']

Reviews

In [390]:
reviews = soup_1.find_all('li', {'class': 'margin-b5__373c0__3ho0z'})

In [391]:
len(reviews)

10

In [401]:
reviews[9].find('span', {'class': 'raw__373c0__tQAx6'}).get_text(' ')

"Wow! This was our first stop after we landed in Seattle (after checking into our hotel and dropping off our bags, lol). We wanted to try Pink Door but was unable to find a reservation even a few weeks prior. We decided to call and the lady on the phone said they accept walk-ins at 5PM, but first come first serve.  Note: This was on a Thursday, so policies might be different on a weekend. We got to the restaurant around 4:45PM, and there was one other party waiting in line already. We stuck around and the walk-up line grew immensely. They have two separate lines, one for reservations, and one for walk-ins. The hostess sat all the reservations first, and then walk-ins starting at 5PM. Thus, although we were second in line, we did not get seated until like 5:20. I think most people walking up got seated because the restaurant was probably only half filled by the time we got in. We decided to order the lasagna pink door and the Cioppino Pink Door to share between the two of us. The Lasagn

### Find all the Restaurants URLs

In [48]:
# create dataframe (in the beginning it is empty)
Restaurants_Seattle_Hours = pd.DataFrame(columns=['Name', 'Address', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])


for i in range(24, 25):
    # website request
    website = 'https://www.yelp.com/search?find_desc=restaurants&find_loc=Federal%20Way%2C%20WA&start=' + str(i) + '0'
    response = requests.get(website)

    # create soup object
    soup = bs(response.content, 'html.parser')

    # result container
    result_container = soup.find_all('div', {'class':'container__09f24__sxa9-'})
    
    # URL part 1
    url_part_1 = 'https://www.yelp.com/'
    
    # URL part 2
    url_part_2 = []

    # loop through results
    for item in result_container:
        # Loop through links
        for link in item.find_all('a', {'class': 'css-166la90'}):
            url_part_2.append(link.get('href'))
    
    # join URL 1 and URL 2 
    url_joined = []

    for link_2 in url_part_2:
        url_joined.append(urllib.parse.urljoin(url_part_1, link_2))
    
    hour = []
    # loop through all joined links
    for link in url_joined:
        response = requests.get(link)

        # create soup object
        soup = bs(response.content, 'html.parser')

        #name
        try:
            name = soup.find('h1', {'class': 'css-11q1g5y'}).get_text()
        except:
            name = 'n/a'

        #address
        try:
            address = soup.find('address').get_text(", ")
        except:
            address = 'n/a'
            
        #hours
        try:
            hours = soup.find('tbody').find_all('tr')[1::2]
            hour = []
            for tr in hours:
                td = tr.find_all('td')
                data = [str(tr.get_text(", ")).strip('') for tr in td]
                hour.append(data)
           
        except:
            hour = 'n/a',
            
            
        Restaurants_Seattle_Hours = Restaurants_Seattle_Hours.append({'Name': name,
                                                                      'Address': address,
                                                                      'Mon': hour[0],
                                                                      'Tue': hour[1],
                                                                      'Wed': hour[2],
                                                                      'Thu': hour[3],
                                                                      'Fri': hour[4],
                                                                      'Sat': hour[5],
                                                                      'Sun': hour[6]
                                                                      },
                                                                       ignore_index=True)

In [49]:
Restaurants_Seattle_Hours

,Name,Address,Mon,Tue,Wed,Thu,Fri,Sat,Sun


In [461]:
#Restaurants_Seattle_Hours.to_csv('Restaurants_Seattle_Hours.csv', index=False, encoding='utf-8-sig', na_rep='Unknown')

In [47]:
with open('Restaurants_Seattle_Hours.csv', 'a') as f:
    Restaurants_Seattle_Hours.to_csv(f, header=False, index=False, encoding='utf-8-sig')

# Reviews